In [186]:
!pip install pygsheets

import pygsheets
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [191]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [192]:
def get_gs_client():
    c = pygsheets.authorize(service_account_file='nota-climat-0d4d93fe7916.json')
    return c

def load_gsheet_in_df(gs_id, tab_name):
    c = get_gs_client()
    spreadsheet = c.open_by_key(gs_id)
    wks = spreadsheet.worksheet('title', tab_name)
    df = pd.DataFrame(wks.get_all_records())
    return df

def write_df_gsheet(gs_id, tab_name, df):
    c = get_gs_client()
    spreadsheet = c.open_by_key(gs_id)
    wks = spreadsheet.worksheet('title', tab_name)
    wks.set_dataframe(df, 'A1')
    return df

In [193]:
#loading data from scope 1 & 2 analysis spreadsheet document
df = load_gsheet_in_df(gs_id='1L9fm760u7GC8-L2oiNcr1zTudxuKBkUdgJTrZpiAqWg',
                                        tab_name='last_version_morgane')

In [194]:
scope_3 = df[df["scope"]==1][["Company Name", 'Target', 'target_year', 'base_year']].drop_duplicates()
scope_3.rename(columns={'target_year': 'target_year_scope1',
                                  'base_year': 'base_year_scope1'}, inplace=True)

In [195]:
#Target in lowercase
scope_3["Target"]=scope_3["Target"].str.lower()

#replacing all 'fy{YEAR}' by '{YEAR}'
scope_3["Target"]=scope_3["Target"].str.replace(' fy', '') 

#replacing all 'by{YEAR}' by 'by {YEAR}'
scope_3["Target"]=scope_3["Target"].str.replace(' by20', ' by 20') 

#replacing all 'from a{YEAR}' by 'from a {YEAR}'
scope_3["Target"]=scope_3["Target"].str.replace('from a20', 'from a 20') 

In [196]:
len(scope_3)

1218

In [197]:
#creating the expected columns to fill
scope_3["scenario"]=''
scope_3["details_perimeter"] = ""
scope_3["unit"] = ""
scope_3["unit_absolute"] = ""
scope_3["%_target"] = ""
scope_3["target_year"] = ""
scope_3["base_year"] = ""

# 1st scenario 
Removing those with "measure reduce scope 3 emissions"

In [198]:
scope_3["scenario"]=np.where((scope_3["scenario"]=='')\
                             & scope_3["Target"].str.contains('and to measure and reduce its scope 3 emissions'),
                            "1",
                            scope_3["scenario"])


scope_3["details_perimeter"] = np.where((scope_3["scenario"]=='1'),
                                        'measure and reduce its scope 3 emissions',
                                        scope_3["scenario"])

In [199]:
scope_3[scope_3["scenario"]=='1'].head(1)

,Company Name,Target,target_year_scope1,base_year_scope1,scenario,details_perimeter,unit,unit_absolute,%_target,target_year,base_year
0,(ACIP) Alexandria Company for Industrial Packages,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ (acip) alexandria company for industrial packages commits to reduce scope 1 and scope 2 ghg emissions 25% by 2030 from a 2020 base year, and to measure and reduce its scope 3 emissions.",2030,2020,1,measure and reduce its scope 3 emissions,,,,,


In [200]:
print("remaining scenarios : {}".format(len(scope_3[scope_3["scenario"]==''])))

remaining scenarios : 905


# 2nd scenario

Example : commits to reduce scope 3 GHG emissions 67% per square meter within the same timeframe

In [201]:
pattern2_groups = r'( absolute)* scope( 1, 2,? and)? 3(.*) ([0-9]+.?[0-9]+?\ ?%)(.*)((within|over) .* (timeframe|target period|time-period))'

In [202]:
scope_3["scenario"]=np.where((scope_3["scenario"]=='')\
                             & scope_3["Target"].str.extract(pattern2_groups)[2].notnull(),
                            "2",
                            scope_3["scenario"])

In [203]:
scope_3["unit_absolute"] = np.where((scope_3["scenario"]=='2'),
                                        scope_3["Target"].str.extract(pattern2_groups)[0],
                                        scope_3["%_target"])

scope_3["details_perimeter"] = np.where((scope_3["scenario"]=='2'),
                                        scope_3["Target"].str.extract(pattern2_groups)[2],
                                        scope_3["details_perimeter"])

scope_3["%_target"] = np.where((scope_3["scenario"]=='2'),
                                        scope_3["Target"].str.extract(pattern2_groups)[3],
                                        scope_3["%_target"])

scope_3["unit"] = np.where((scope_3["scenario"]=='2'),
                                        scope_3["Target"].str.extract(pattern2_groups)[4],
                                        scope_3["%_target"])

scope_3["target_year"] = np.where((scope_3["scenario"]=='2'),
                                        scope_3["Target"].str.extract(pattern2_groups)[5],
                                        scope_3["target_year"])

scope_3["base_year"] = np.where((scope_3["scenario"]=='2'),
                                        scope_3["Target"].str.extract(pattern2_groups)[5],
                                        scope_3["base_year"])


In [204]:
scope_3[scope_3["scenario"]=='2'].head(1)

,Company Name,Target,target_year_scope1,base_year_scope1,scenario,details_perimeter,unit,unit_absolute,%_target,target_year,base_year
53,Morrisons,"morrisons commits to reduce absolute scope 1 and 2 ghg emissions 46% by 2030 from a 2019 base year. morrisons also commits to reduce absolute scope 3 ghg emissions from own-brand purchased goods and services, own-brand upstream transportation and distribution and use of sold products 30% over the same timeframe.",2030,2019,2,"ghg emissions from own-brand purchased goods and services, own-brand upstream transportation and distribution and use of sold products",,absolute,30%,over the same timeframe,over the same timeframe


In [205]:
print("remaining scenarios : {}".format(len(scope_3[scope_3["scenario"]==''])))

remaining scenarios : 652


# 3rd scenario

In [206]:
pattern3_groups = r'( absolute)* scope( 1, 2,? and)? 3(.*) ([0-9]+.?[0-9]+?\ ?%)(.*)by ([0-9]+) from a ([0-9]+) base.year\. (.*)'

In [207]:
scope_3["scenario"]=np.where((scope_3["scenario"]=='')\
                             & scope_3["Target"].str.extract(pattern3_groups)[2].notnull(),
                            "3",
                            scope_3["scenario"])

In [208]:
scope_3["unit_absolute"] = np.where((scope_3["scenario"]=='3'),
                                        scope_3["Target"].str.extract(pattern3_groups)[0],
                                        scope_3["unit_absolute"])

scope_3["details_perimeter"] = np.where((scope_3["scenario"]=='3'),
                                        scope_3["Target"].str.extract(pattern3_groups)[2],
                                        scope_3["details_perimeter"])

scope_3["%_target"] = np.where((scope_3["scenario"]=='3'),
                                        scope_3["Target"].str.extract(pattern3_groups)[3],
                                        scope_3["%_target"])

scope_3["unit"] = np.where((scope_3["scenario"]=='3'),
                                        scope_3["Target"].str.extract(pattern3_groups)[4],
                                        scope_3["unit"])

scope_3["target_year"] = np.where((scope_3["scenario"]=='3'),
                                        scope_3["Target"].str.extract(pattern3_groups)[5],
                                        scope_3["target_year"])

scope_3["base_year"] = np.where((scope_3["scenario"]=='3'),
                                        scope_3["Target"].str.extract(pattern3_groups)[6],
                                        scope_3["base_year"])


In [209]:
scope_3[scope_3["scenario"]=='2'].head(1)

,Company Name,Target,target_year_scope1,base_year_scope1,scenario,details_perimeter,unit,unit_absolute,%_target,target_year,base_year
53,Morrisons,"morrisons commits to reduce absolute scope 1 and 2 ghg emissions 46% by 2030 from a 2019 base year. morrisons also commits to reduce absolute scope 3 ghg emissions from own-brand purchased goods and services, own-brand upstream transportation and distribution and use of sold products 30% over the same timeframe.",2030,2019,2,"ghg emissions from own-brand purchased goods and services, own-brand upstream transportation and distribution and use of sold products",,absolute,30%,over the same timeframe,over the same timeframe


In [210]:
print("remaining scenarios : {}".format(len(scope_3[scope_3["scenario"]==''])))

remaining scenarios : 513


# 4th scenario

In [224]:
pattern4_groups = r'( absolute)* scope( 1, 2,? and)? 3(.*) ([0-9]+.?[0-9]+?\ ?%)(.*)by ([0-9]+),? from a ([0-9]+) base.year'

scope_3["scenario"]=np.where((scope_3["scenario"]=='')\
                             & scope_3["Target"].str.extract(pattern4_groups)[2].notnull(),
                            "4",
                            scope_3["scenario"])

In [225]:
scope_3["unit_absolute"] = np.where((scope_3["scenario"]=='4'),
                                        scope_3["Target"].str.extract(pattern4_groups)[0],
                                        scope_3["unit_absolute"])

scope_3["details_perimeter"] = np.where((scope_3["scenario"]=='4'),
                                        scope_3["Target"].str.extract(pattern4_groups)[2],
                                        scope_3["details_perimeter"])

scope_3["%_target"] = np.where((scope_3["scenario"]=='4'),
                                        scope_3["Target"].str.extract(pattern4_groups)[3],
                                        scope_3["%_target"])

scope_3["unit"] = np.where((scope_3["scenario"]=='4'),
                                        scope_3["Target"].str.extract(pattern4_groups)[4],
                                        scope_3["unit"])

scope_3["target_year"] = np.where((scope_3["scenario"]=='4'),
                                        scope_3["Target"].str.extract(pattern4_groups)[5],
                                        scope_3["target_year"])

scope_3["base_year"] = np.where((scope_3["scenario"]=='4'),
                                        scope_3["Target"].str.extract(pattern4_groups)[6],
                                        scope_3["base_year"])


In [226]:
final_df = scope_3
final_df["unit"]=np.where(final_df["unit_absolute"]==' absolute', 'absolute', final_df["unit"])
final_df = final_df.drop(columns=["unit_absolute"])

In [227]:
# Add specificities to identify next steps
final_df["specificity"] = np.where(final_df.Target.str.contains(pat=' 3 .* 3 '), 'Contains several scopes 3', 'None')
final_df["specificity"] = np.where(final_df.Target.str.contains(pat='by 20.. and .?..% by 20..'), 'Has several targets year', final_df["specificity"])

In [228]:
# replacing 'over the same timeframe' expressions in both target and base year
# by the actual timeframe from scope 1 data.
same_timeframe_expressions = ['over the same timeframe',
                              'within the same timeframe',
                              'over the same target period',
                              'over the same time-period',
                              'over the same target timeframe']

final_df["target_year"] = np.where(final_df["target_year"].isin(same_timeframe_expressions), 
                                  final_df["target_year_scope1"],
                                  final_df["target_year"])

final_df["base_year"] = np.where(final_df["base_year"].isin(same_timeframe_expressions), 
                                   final_df["base_year_scope1"],
                                   final_df["base_year"])

In [230]:
final_df.drop(columns=['target_year_scope1', 'base_year_scope1'], inplace=True)

In [231]:
write_df_gsheet(gs_id='1L9fm760u7GC8-L2oiNcr1zTudxuKBkUdgJTrZpiAqWg',
                tab_name='output David',
                df=final_df)

,Company Name,Target,scenario,details_perimeter,unit,%_target,target_year,base_year,specificity
0,(ACIP) Alexandria Company for Industrial Packages,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ (acip) alexandria company for industrial packages commits to reduce scope 1 and scope 2 ghg emissions 25% by 2030 from a 2020 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None
2,AM Værktøj Odense A/S,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ am værktøj odense a/s commits to reduce scope 1 and scope 2 ghg emissions 50% by 2030 from a 2018 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None
4,Julie Sandlau Vietnam. ltd,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ julie sandlau vietnam. ltd commits to reduce scope 1 and scope 2 ghg emissions 42% by 2030 from a 2020 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None
6,Kartikeya International,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ kartikeya international commits to reduce scope 1 and scope 2 ghg emissions 50% by 2030 from a 2018 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None
8,PlanA.Earth GmbH,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ plana.earth gmbh commits to reduce scope 1 and scope 2 ghg emissions 42% by 2030 from a 2020 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None
...,...,...,...,...,...,...,...,...,...
3047,"Simões, Lda","this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ simões, lda commits to reduce scope 1 and scope 2 ghg emissions 50% by 2030 from a 2018 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None
3049,Statik,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ statik commits to reduce scope 1 and scope 2 ghg emissions 50% by 2030 from a 2018 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None
3051,U&We AB,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ u&we ab commits to reduce scope 1 and scope 2 ghg emissions 46% by 2030 from a 2019 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None
3053,WindowMaster International A/S,"this target was approved using a streamlined target validation route exclusive to small and medium-sized enterprises (smes). https://sciencebasedtargets.org/faqs-for-smes/ windowmaster international a/s commits to reduce scope 1 and scope 2 ghg emissions 46% by 2030 from a 2019 base year, and to measure and reduce its scope 3 emissions.",1,measure and reduce its scope 3 emissions,,,,,None


NEXT STEPS:
*   when "details_perimeter" contains 'ghg emissions 20% by 2025 and' -> to put in another columns with specific target 2 values
*   investigate some complex cases like Adidas AG